<a href="https://colab.research.google.com/github/AmiJoba22/RateMD-Datapipe/blob/main/pythonscripts%20/%20extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Get data from source to put in destination
#Destination has an address; source has an address
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [7]:
# read the config file that is json
import json
with open('config.json') as config_file:
  config = json.load(config_file)

#read information about the destination
DESTINATION_CONNECTION_STRING = config['DESTINATION_CONNECTION_STRING']
DESTINATION_CONTAINER_NAME = config['DESTINATION_CONTAINER_NAME']
DESTINATION_FILE_NAME = config ['DESTINATION_FILE_NAME']

# read information about mongodb
MONGODB_CONNECTION_STRING = config ['MONGODB_CONNECTION_STRING']
MONGO_DB_NAME = config ['MONGO_DB_NAME']
MONGO_COLLECTION_NAME = config ['MONGO_COLLECTION_NAME']
print(MONGO_COLLECTION_NAME)

ratemd


In [8]:
# connect to mongodb and test the connection using ping
client = MongoClient(MONGODB_CONNECTION_STRING)
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
db = client[MONGO_DB_NAME]
collection = db[MONGO_COLLECTION_NAME]
print(collection)

Pinged your deployment. You successfully connected to MongoDB!
Collection(Database(MongoClient(host=['20.232.135.212:27017'], document_class=dict, tz_aware=False, connect=True, authmechanism='DEFAULT', authsource='admin'), 'healthrate'), 'ratemd')


In [3]:
# read the collection and store it into azure blob
# Retrieve all documents from the collection
mongo_data = list(collection.find())

# Convert to pandas DataFrame (optional, but useful for many formats)
df = pd.DataFrame(mongo_data)

# Choose a file format and save the data
# For example, saving to CSV:
local_file_name = "mongo_collection_data.csv"
df.to_csv(local_file_name, index=False)

# Upload to Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(DESTINATION_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(DESTINATION_CONTAINER_NAME)

with open(local_file_name, "rb") as data:
    blob_client = container_client.upload_blob(name=DESTINATION_FILE_NAME, data=data, overwrite=True)

print(f"Data from MongoDB collection '{MONGO_COLLECTION_NAME}' uploaded to Azure Blob '{DESTINATION_FILE_NAME}' in container '{DESTINATION_CONTAINER_NAME}'.")

NameError: name 'collection' is not defined